## NER results to network mapping 
- Results from NER require cleaning 
- Dictionary to define what is picked up by the model to what it actually means 
- And what organisation the entity is related to 
- Then need to label the type of entity beyond org/person to the type of org it is (describe the entity)
- This is to map the network in neo4j and/or vistorian

In [1]:
import pandas as pd
results=pd.read_csv('all_entities_myanmar.csv').drop(columns=['Unnamed: 0'])

- Can see from the df below that row 0 and row 3 both mean CNF, but cannot tell from this
- Need to map the entities recognised to what they really mean

In [2]:
results.head()

,doc_id,agreement_id,sentence_number,entity,entity_type
0,0,1321,0,CNF - Govt Agreements,ORG
1,0,1321,0,Chinland Guardian,ORG
2,0,1321,0,Chin National ( CNF,ORG
3,0,1321,0,Peace Committee Burma,ORG
4,0,1321,0,Rangoon,GPE


In [3]:
entity_dict={
    'Chin National Front (CNF)':[' CNF - Govt Agreements',' Chin National ( CNF', ' CNF', ' Chin National Chin State ',  ' Chin National / Chin National Army', ' Chin', ' Chin National Day', ' Chin National Peace Team', ' CNA',  ' Chin National ’s', ' CNF Government',  ' Chin National / Army', ' Chin National Union', ' Chin National Chin State / Union Government', ' Chin National Peace Negotiating Team', ' Chin State Government', ' Government Chin State', ' Chin Special Division', ' Chin State Union Government', ' Chin National Peace Team', ' Chin National Chin State / Union Government', ' Chin National', ' Chin National Chin National Army', ' Chin National Army'],
    'Joint Monitoring Committee (JMC)':[' Joint Monitoring Committee', ' JMC', ' Joint Monitoring Committee Guideline'],
    'Joint Ceasefire Monitoring Committee (JCMC)': [ ' Joint Ceasefire Monitoring Committee', ' Joint Ceasefire Monitoring',' Ceasefire Joint Monitoring Committee', ' Union Level Joint Ceasefire Monitoring Committee', ' Local Level Joint Ceasefire Monitoring Committee',' Nationwide Ceasefire Agreement Joint Implementation Coordination Meeting'],
    'Union Level Joint Ceasefire Monitoring Committee (JMC-U)':[' JMC - U', ' JMC - U.  Meetings Resolutions', ' Union - Level', ' JMC - U. ',' Union Peace Dialogue Joint Committee'],
    'State Level Joint Ceasefire Monitoring Committee (JMC-S)':[' JMC - S', ' JMC - S.', ' JMC - S. '],
    'Local Level Joint Ceasefire Monitoring Committee (JMC-L)':[' JMC - L', ' JMC - L.  Responsibilities JMC - L'],
    'Joint Implementing Coordination Meeting (JICM)':[' JICM',],
    'Karen National Union (KNU)/Karen National Liberation Army (KNLA)':['Karen National Union', ' KNLA  Permanent Central Committee', ' KNU', ' KNU / KNLA', ' KNU Press Release', ' KNU Delegation', ' Karen National Union Foreign & Public Relations Department', ' KNU / KNLA Peace Council 9',  ' KNLA Peace Council', ' KNU / KNLA Peace Council', ],
    'Karenni National Progressive Party (KNPP)':[' Karenni National Progressive Party', ' KNPP', ' KNPP Khu Oo Reh'],
    'Union Peace Dialogue Joint Committee (UPDJC)':[' UPDJC', ' Union Peace Working Committee', ' Union Peace - Making Work Committee', ' Union Peace Dialogue Joint Committee', ' Union Peace',  ' Union Peace Team Crisis Mediating Body'],
    'Government of the Republic of the Union of Myanmar':[' Union Government', ' Government Republic Union Myanmar', ' Government ’s Order',' Government Group', ' Myanmar Government', ' Government Burma',' Burmese Government',],
    'Hluttaw':[' Hluttaw', ' Hluttaws', ' Hluttaw Group  3', ' Pyidaungsu Hluttaw'],
    'Tatmadaw':[' Tatmadaw', ' Pyidaungsu Tatmadaw', ' Tatmadaw Group  4'],
    'Shan State Progress Party (SSPP)/Shan State Army (SSA)':[' SSA - S Government Ceasefire Agreement',' SSA - S Government 11',' SSA', ' SSA - S',' Restoration Council Shan State / Shan State Army',' SSPP / SSA - N Government  Preliminary Peace', ' Shan State Progressive Party', ' SSPP)/ Shan State Army',' Shan State',' SSA - S Government',  ' Shan State Minster Security', ' SSPP)/ Shan State Army', ' SSPP / Shan State Army Sao Khun Hsai',  ' SSPP / Shan State Army', ' Shan State Progressive',  ' Party/ Shan State Army', ' Shan State Police Force', ' Mongla Armed Group ( Special Region 4 - Northern Shan State',' Shan State Progressive Party / Shan State Army',],
    'Arakan Army':[ ' Arakan Liberation Party Arakan National Council', ' General Arakan Army Chin National'],
    'Rakhine State Liberation Party (RSLP)':[ ' Rakhine State Liberation Party', ' Rakhine State Liberation Party (RSLP)'],
    'Ethnic Armed Organizations (EAO)': [ ' Ethnic Armed Organizations', ' Ethnic Armed  Organizations',' Ethnic Armed Organization', ' Ethnic Armed Organization Group  5', ' Ethnic Armed Organization Group  U Zein Khan Ram  Ethnic Representative Political Parties Group', ' ee ) EAOs ', ' EAOs '],
    'Food and Drug Administration (FDA)': [' FDA'], 
    'National League Democracy (NLD)':[' NLD'],
    'National Unity Party (NUP)': [' NUP'],
    'United Nationalities Alliance (UNA)':[' UNA',],
    'National Democratic Alliance Army (NDAA)':[' NDAA',],
    'Lahu Democratic Union':[ ' Lahu Democratic Union Myanmar National Democratic Alliance Army',],
    'United Nations':[' U.N.', ' UN Office',],
    'Pa-O National Liberation Organisation (PNLO)': [' PNLO'],
    'Democratic Karen Benevolent/Buddhist Army (DKBA)':[ ' Democratic Karen Benevolent Army', ' Democratic Karen Benevolent Army', ' Democratic Karen Benevolent Army', ],
    'United Nations Office on Drugs and Crime (UNODC)':[' UNODC'],
    'Union Peacemaking Working Committee (UPWC)':[' UPWC'],
    'New Mon State Party (NMSP)': [' NMSP', ' New Mon State Party', ' NMSP CEC', ' NMSP Government'],
    'Union Election Commission (UEC)': [' UEC'],
    'Kachin Independence Organisation (KIO)': [' KIO',' Kachin Independence Organization'],
    'United Nationalities Federal Council (UNFC)': [' UNFC'],
    'National Socialist Council of Nagaland (NSCN)':[' Naga', ' NSCN', ' NSCN-K', ' NSCN (K)', ' NSCN - K',],
    'All Burma Students Democratic Front (ABSDF)': [' ABSDF Union', ' ABSDF'],
    'Nationalities Brotherhood Federation (NBF)': [' NBF'],
    'General Aung San':[' Aung San'],
    'Lt. Gen. Ya Pyit':[' Gen. Ya Pyit'],
    'U Khin Maung Soe': [' U Khin Maung Soe  Union'],
    'Lt. Gen. Ye Aung':[' Gen. Ye Aung  Chief'],
    'Maj. Gen. Tun Tun Naung':[' Tun Tun Naung',],
    'Maj. Gen. Aung Kyaw Zaw': [' Aung Kyaw Zaw', ' North - East Command Maj - Gen Aung Kyaw Zaw'],
    'U Myint Soe':[' U Myint Soe  Retired Lt'],
    'Maj. Gen. Aung Soe':[' Aung Soe', ' Aung Soe ',],
    'Col. Wun Na Aung':[' Ministry Interior'],
    'Dr. Min Zaw Oo':[' Min Zaw'],
    'Maj. Gen. Saw Isaac Poe':[ ' Gen. Saw Isaac Poe  Quartermaster General',],
    'Maj. Gen. Saw Nay Soe Mya':[' Gen. Saw Nay Soe Mya  Adjutant General',],
    'Dr. Shwe Kah':[ ' Shwe Kah',],
    'Col. Saw Kyaw Than Htay':[' Saw Kyaw Htay'],
    'Lt. Col. Khun Aung Mang':[' Khun Aung Mang  PNLO',],
    'Salai Yaw Aung':[' Salai Yaw Aung  ABSDF'],
    'Col. Hsaung Han':[' Hsaung Han',],
    'Sai Liang':[' Sai Liang '],
    'U Aung Min':[' U Aung Min'],
    'Khu Oo Reh':[' KNPP Khu Oo Reh',],
     'U Saw Mu Shay': [' Kalo Htoo Baw U Saw Mu Shay',],
    'Col. Aung Lwin':[' Aung Lwin', ],
    'Maj. Gen Saw Lar Pwe':[' Gen Saw Lar Pwe',],
    'U Aung Thaung':[' U Aung Thaung',],
    'U Thein Zaw':[' U Thein Zaw',],
    'U Ohn Myint':[' U Ohn Myint'],
    'U Win Tun':[' U Win Tun',],
    'U Sao Aung Myat':[' U Sao Aung Myat'],
    'U Maung Maung Thein':[' U Maung Maung Thein'],
    'Maj-Gen Aung Kyaw Zaw':[' North - East Command Maj - Gen Aung Kyaw Zaw'],
    'Maj-Gen Mya Tun Oo':[' Central East Command'],
    'Maj-Gen Zaw Win':[' Maj - Gen Zaw Win'],
    'U Soe Tint':[' U Soe Tint',],
    'Col Aung Thu.':[' Aung Thu',],
    'General Sao Khay Tai':[' Sao Khay Tai',],
    'Sao Khin Min':[ ' Sao Khin Min',],
    'Sao Khun Hsai':[' Sao Khun Hsai'],
    'Sao Khay Maung':[' Sao Khay Maung'],
    'Sao Kyaw Hla':[' Sao Kyaw Hla'],
    'Sao Htin Mein':[' Sao Htin Mein'],
    'Sao Laung Khay':[' Sao Laung Khay'],
    'Sao Naw Leik':['  Sao Naw Leik'],
    'General Secretary Naw Zipporah Sein':[' Naw Zipporah Sein',],
    'U Saw Lah Pwe':[ ' U Saw Lah Pwe',],
    'U Khun Okkar':[' Khun Okkar'],
    'U Khun Myint Htun': [' U Khun Myint Htun'],
    'U Khin Mg Soe':[' Khin Mg Soe'],
    'U Twan Zaw':[' U Twan Zaw'],
    'U Tun Mrat Naing':[' U Tun Mrat Naing'],
    'Pu Zin Cun':[' Pu Zin Cun Commander',],
    'Gen. Saw Mo Shay':[' Saw Mo Shay',],
    'Koon Abel Tweed':[' Koon Abel Tweed',],
    'Col. Saw Kyaw Dah':[' Saw Kyaw Dah',],
    'Kya Khunsa':[' Kya Khunsa',],
    'Dr. Tin Myo Win':[' Tin Myo Win  Representative',],
    'Dr. May Win Myint':[' Win Myint  Representative',],
    'U Thu Wai':[' U Thu Wai  Representative'],
    'U Kyaw Win':[' Kyaw Win  Relevant Representative'],
    'U Zein Khan Ram':[' Ethnic Armed Organization Group  U Zein Khan Ram  Ethnic Representative Political Parties Group'],
    'U Thein Sein':[' Union Myanmar U Thein Sein'],
    'Pu Zing Cung':[' Pu Zing Cung'],
    'Dr. Sui Khar':[' Sui Khar', ' Joint General',],
    'U Win Htun':[' U Win Htun',],
    'U Hung Ngai':[' U Hung Ngai',],
    'Htein Lin':[' Htein Lin'],
    'Y Wantin Naga':[' Y Wantin Naga'],
    'Maj - Gen Tin Maung Win':[' South - East Command Maj - Gen Tin Maung Win',],
    'Myanmar (Burma)':[' Burma', ' Myanmar', ' Union Burma',
        ' Union',  ' Republic Union Myanmar', ' Republic Myanmar', ' Union Republic Myanmar', ' Republic Union Burma', ' Union Myanmar',]
    
}

- Function to apply to entity columns - if in dictionary, return key:

In [4]:
def label_ent(entity_name):
    for key, value in entity_dict.items():
        if entity_name in value: 
            return key   

In [5]:
results['corrected_entity']=results['entity'].apply(label_ent)

- Not all need corrected, so those not defined in dictionary need to be filled in new column

In [6]:
def label_ner(s):
    if (s['corrected_entity'] == None):
        return (s['entity'])
    else:
        return (s['corrected_entity'])

In [7]:
results['new_entity']=results.apply(label_ner, axis=1)

In [8]:
results.head()

,doc_id,agreement_id,sentence_number,entity,entity_type,corrected_entity,new_entity
0,0,1321,0,CNF - Govt Agreements,ORG,Chin National Front (CNF),Chin National Front (CNF)
1,0,1321,0,Chinland Guardian,ORG,None,Chinland Guardian
2,0,1321,0,Chin National ( CNF,ORG,Chin National Front (CNF),Chin National Front (CNF)
3,0,1321,0,Peace Committee Burma,ORG,None,Peace Committee Burma
4,0,1321,0,Rangoon,GPE,None,Rangoon


## Dictionary for affiliated organisations
- Useful to know what organisations these entities represent 
- Some are organisations themselves, others are people representing an organisation

In [9]:
person_affilitation={
    'Government of the Republic of the Union of Myanmar':[' Htein Lin',' U Win Htun',' Union Myanmar U Thein Sein',' Kyaw Win  Relevant Representative', 'U Soe Tint', ' Win Myint  Representative',' Tin Myo Win  Representative',' Aung San', ' Gen. Ya Pyit', ' U Khin Maung Soe  Union',' U Soe Thein', ' U Zaw Min', ' Gen. Ye Aung  Chief',' Tun Tun Naung', ' Aung Kyaw Zaw', ' U Myint Soe  Retired Lt', ' Aung Soe', ' Aung Soe ', ' Ministry Interior', ' Min Zaw', ' U Aung Min', ' Gen Saw Lar Pwe', ' U Aung Thaung', ' U Thein Zaw', ' U Ohn Myint', ' U Win Tun', ' U Maung Maung Thein', ' North - East Command Maj - Gen Aung Kyaw Zaw',' Central East Command',' Maj - Gen Zaw Win', ],
    'Karen National Union (KNU)/Karen National Liberation Army (KNLA)':[' Saw Kyaw Dah',' Gen. Saw Isaac Poe  Quartermaster General', ' Gen. Saw Nay Soe Mya  Adjutant General', ' Naw Zipporah Sein',],
    'Chin National Front (CNF)':[' Shwe Kah', ' Pu Zin Cun Commander',' Pu Zing Cung', ' Sui Khar', ' Lian Hmung Sakhong', ],
    'Democratic Karen Benevolent/Buddhist Army (DKBA)':[' Saw Kyaw Htay', ' Saw Mo Shay', 'Col. Saw Kyaw Than Htay'],
    'Pa-O National Liberation Organisation (PNLO)':[' Khun Aung Mang  PNLO', 'U Khun Myint Htun',' Khun Okkar', 'U Khun Myint Htun'],
    'All Burma Students Democratic Front (ABSDF)':[' Salai Yaw Aung  ABSDF'],
    'Restoration Council of Shan State (RCSS)':[' Hsaung Han',' Sai Liang '],
    'Karenni National Progressive Party (KNPP)':[' KNPP Khu Oo Reh', ' Koon Abel Tweed',],
    'Kalo Htoo Baw faction of Democratic Karen Benevolent Army (DKBA-5)':[' Kalo Htoo Baw', ' Kaloh Htoo Baw', ' Kalo Htoo Baw U Saw Mu Shay', ' U Saw Lah Pwe'],
    'Karen State Government':[' Aung Lwin',],
    'Shan State Government':[' U Sao Aung Myat', ' Aung Thu', ' Shan State Minster Security'],
    'Chin State Government':[' U Hung Ngai',],
    'Shan State Progress Party (SSPP)/Shan State Army (SSA)':[ ' Sao Khay Tai',' Sao Naw Leik', ' Sao Kyaw',  ' Sao Khin Min', ' Sao Khun Hsai', ' Sao Khay Maung',' Sao Kyaw Hla',' Sao Htin Mein',' Sao Laung Khay','  Sao Naw Leik'],
    'Union Peacemaking Working Committee (UPWC)':[' Khin Mg Soe'],
    'Arakan National Council':[' U Twan Zaw'],
    'Arakan Army':[' U Tun Mrat Naing'],
    'Lahu Democratic Union':[' Kya Khunsa'],
    'Political Parties Group':[' U Thu Wai  Representative', ' Ethnic Armed Organization Group  U Zein Khan Ram  Ethnic Representative Political Parties Group'],
    'National Socialist Council of Nagaland (NSCN)':[' Y Wantin Naga'],
    'Rakhine State Liberation Party (RSLP)':[' Rakhine State Peace Discussion Group'],
    'Special Region (4)':[' Sai Lin',' U Kham Maung',],
    'Tatmadaw':[' South - East Command Maj - Gen Tin Maung Win', ' Col Htay Myint Aung',],
    'New Mon State Party (NMSP)':[' U Nai Rawsa', ' U Nai Talanyi',],
    'Wa Special Region (2) Peacemaking Group':[' U Shauk Min Lyan'], 
}

In [10]:
def match_affiliation_to_key(affiliation_name):
    for key, value in person_affilitation.items():
        if affiliation_name in value:
            return key

In [11]:
results['entity_affiliation']=results['entity'].apply(match_affiliation_to_key)

In [12]:
def affiliation(s):
    if (s['entity_affiliation']== None):
        return (s['new_entity'])
    else: 
        return (s['entity_affiliation'])

In [13]:
results['affiliated_org']=results.apply(affiliation, axis=1)

In [14]:
results

,doc_id,agreement_id,sentence_number,entity,entity_type,corrected_entity,new_entity,entity_affiliation,affiliated_org
0,0,1321,0,CNF - Govt Agreements,ORG,Chin National Front (CNF),Chin National Front (CNF),None,Chin National Front (CNF)
1,0,1321,0,Chinland Guardian,ORG,None,Chinland Guardian,None,Chinland Guardian
2,0,1321,0,Chin National ( CNF,ORG,Chin National Front (CNF),Chin National Front (CNF),None,Chin National Front (CNF)
3,0,1321,0,Peace Committee Burma,ORG,None,Peace Committee Burma,None,Peace Committee Burma
4,0,1321,0,Rangoon,GPE,None,Rangoon,None,Rangoon
...,...,...,...,...,...,...,...,...,...
1372,35,776,4,Rakhine State Peace Discussion Group,ORG,None,Rakhine State Peace Discussion Group,Rakhine State Liberation Party (RSLP),Rakhine State Liberation Party (RSLP)
1373,35,776,4,State Security,ORG,None,State Security,None,State Security
1374,35,776,4,Htein Lin,PERSON,Htein Lin,Htein Lin,Government of the Republic of the Union of Mya...,Government of the Republic of the Union of Mya...
1375,35,776,5,Rakhine State Liberation Party,ORG,Rakhine State Liberation Party (RSLP),Rakhine State Liberation Party (RSLP),None,Rakhine State Liberation Party (RSLP)


In [15]:
shortened_name_dict={
    'CNF': 'Chin National Front (CNF)',
    'JMC':  'Joint Monitoring Committee (JMC)',
    'JICM':'Joint Implementing Coordination Meeting (JICM)',
    'JCMC':'Joint Ceasefire Monitoring Committee (JCMC)',
    'JMC-U':'Union Level Joint Ceasefire Monitoring Committee (JMC-U)',
    'JMC-S':'State Level Joint Ceasefire Monitoring Committee (JMC-S)',
    'JMC-L':'Local Level Joint Ceasefire Monitoring Committee (JMC-L)',
    'KNU/KNLA':'Karen National Union (KNU)/Karen National Liberation Army (KNLA)',
    'KNPP':'Karenni National Progressive Party (KNPP)',
    'UPDJC':'Union Peace Dialogue Joint Committee (UPDJC)',
    'Gov. of Myanmar':'Government of the Republic of the Union of Myanmar',
    'Hluttaw':'Hluttaw',
    'Tatmadaw':'Tatmadaw',
    'SSPP/SSA':['Shan State Progress Party (SSPP)/Shan State Army (SSA)', 'SSPP', 'SSPP/SSA'],
    'AA':'Arakan Army',
    'RSLP':'Rakhine State Liberation Party (RSLP)',
    'EAO':'Ethnic Armed Organizations (EAO)',
    'FDA':'Food and Drug Administration (FDA)',
    'NLD':'National League Democracy (NLD)',
    'UNA':'United Nationalities Alliance (UNA)',
    'LDU':'Lahu Democratic Union',
    'UN':'United Nations',
    'DKBA':'Democratic Karen Benevolent/Buddhist Army (DKBA)',
    'ABSDF':'All Burma Students Democratic Front (ABSDF)',
    'RCSS':'Restoration Council of Shan State (RCSS)',
    'DKBA-5':'Kalo Htoo Baw faction of Democratic Karen Benevolent Army (DKBA-5)',
    'Union Peace Group':'Union level peace-making group',
    'NSCN':'National Socialist Council of Nagaland (NSCN)',
    'PPLO':'Pa-O Peoples Liberation Organization (PPLO)',
    'PNLO':'Pa-O National Liberation Organisation (PNLO)',
    'UNODC':'United Nations Office on Drugs and Crime (UNODC)',
    'UPWC':'Union Peacemaking Working Committee (UPWC)',
    'NMSP':'New Mon State Party (NMSP)',
    'UEC':'Union Election Commission (UEC)',
    'KIO':'Kachin Independence Organisation (KIO)',
    'UNFC':'United Nationalities Federal Council (UNFC)',
    'NBF': 'Nationalities Brotherhood Federation (NBF)',
    'ALP': 'Arakan Liberation Party',
    'RSLP':'Rakhine State Liberation Party (RSLP)',
    'NUP':'National Unity Party (NUP)'
}

In [16]:
def match_shortened_to_key(short_name):
    for key, value in shortened_name_dict.items():
        if short_name in value:
            return key

In [17]:
results['affiliated_org_abr']=results['affiliated_org'].apply(match_shortened_to_key)

In [18]:
results

,doc_id,agreement_id,sentence_number,entity,entity_type,corrected_entity,new_entity,entity_affiliation,affiliated_org,affiliated_org_abr
0,0,1321,0,CNF - Govt Agreements,ORG,Chin National Front (CNF),Chin National Front (CNF),None,Chin National Front (CNF),CNF
1,0,1321,0,Chinland Guardian,ORG,None,Chinland Guardian,None,Chinland Guardian,None
2,0,1321,0,Chin National ( CNF,ORG,Chin National Front (CNF),Chin National Front (CNF),None,Chin National Front (CNF),CNF
3,0,1321,0,Peace Committee Burma,ORG,None,Peace Committee Burma,None,Peace Committee Burma,None
4,0,1321,0,Rangoon,GPE,None,Rangoon,None,Rangoon,None
...,...,...,...,...,...,...,...,...,...,...
1372,35,776,4,Rakhine State Peace Discussion Group,ORG,None,Rakhine State Peace Discussion Group,Rakhine State Liberation Party (RSLP),Rakhine State Liberation Party (RSLP),RSLP
1373,35,776,4,State Security,ORG,None,State Security,None,State Security,None
1374,35,776,4,Htein Lin,PERSON,Htein Lin,Htein Lin,Government of the Republic of the Union of Mya...,Government of the Republic of the Union of Mya...,Gov. of Myanmar
1375,35,776,5,Rakhine State Liberation Party,ORG,Rakhine State Liberation Party (RSLP),Rakhine State Liberation Party (RSLP),None,Rakhine State Liberation Party (RSLP),RSLP


In [19]:
list(results)

['doc_id',
 'agreement_id',
 'sentence_number',
 'entity',
 'entity_type',
 'corrected_entity',
 'new_entity',
 'entity_affiliation',
 'affiliated_org',
 'affiliated_org_abr']

In [20]:
r=results[['agreement_id','entity_type','new_entity', 'affiliated_org',
 'affiliated_org_abr']]

In [21]:
r['NER']='SpaCy'

C:\Users\nhenry2\AppData\Local\Temp/ipykernel_13308/4272546128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['NER']='SpaCy'


In [22]:
r

,agreement_id,entity_type,new_entity,affiliated_org,affiliated_org_abr,NER
0,1321,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
1,1321,ORG,Chinland Guardian,Chinland Guardian,None,SpaCy
2,1321,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
3,1321,ORG,Peace Committee Burma,Peace Committee Burma,None,SpaCy
4,1321,GPE,Rangoon,Rangoon,None,SpaCy
...,...,...,...,...,...,...
1372,776,ORG,Rakhine State Peace Discussion Group,Rakhine State Liberation Party (RSLP),RSLP,SpaCy
1373,776,ORG,State Security,State Security,None,SpaCy
1374,776,PERSON,Htein Lin,Government of the Republic of the Union of Mya...,Gov. of Myanmar,SpaCy
1375,776,ORG,Rakhine State Liberation Party (RSLP),Rakhine State Liberation Party (RSLP),RSLP,SpaCy


## Manual additions
- Not all entities were recognised by the model
- Created a dataset of some people not recognised, to be added in 

In [23]:
manual=pd.read_excel('persons_not_identified.xlsx')

In [24]:
m=manual.drop(columns=['entity'])

In [25]:
list(manual)

['agreement_id',
 'entity',
 'entity_named',
 'entity_type',
 'entity_affiliated_org',
 'entity_affiliated_org_abr',
 'NER']

In [26]:
r1=r.rename(columns={'new_entity':'entity_named','affiliated_org':'entity_affiliated_org',
 'affiliated_org_abr':'entity_affiliated_org_abr'})

In [27]:
both=pd.concat([r1, m])

In [28]:
both

,agreement_id,entity_type,entity_named,entity_affiliated_org,entity_affiliated_org_abr,NER
0,1321,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
1,1321,ORG,Chinland Guardian,Chinland Guardian,None,SpaCy
2,1321,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
3,1321,ORG,Peace Committee Burma,Peace Committee Burma,None,SpaCy
4,1321,GPE,Rangoon,Rangoon,None,SpaCy
...,...,...,...,...,...,...
13,2369,PERSON,U Htone Phoo Dagon,Kachin National Association of Tradition and C...,Gov. of Myanmar,manual
14,2369,PERSON,Kya Sha Mwe,Ethnic Representative Ethnic Armed Organizatio...,EAO,manual
15,2369,PERSON,Daw Khin Ma Ma Myo,Relevant Representative of Ethnic Armed Organi...,EAO,manual
16,2369,PERSON,U Zein Khan Ram,Ethnic Representative Political Parties Group,Political Parties Group,manual


In [29]:
both.to_csv('Myanmar_all_entities_labelled_withM.csv')

## Dictionary to determine what type of entity they are 
- Determine and label if entities are EAO, government groups, NGOs, political parties etc 

In [30]:
type_dict={
    'EAO': ['Ethnic Armed Organizations (EAO)','Chin National Front (CNF)', 'Karen National Union (KNU)/Karen National Liberation Army (KNLA)',
       'Democratic Karen Benevolent/Buddhist Army (DKBA)', 'Pa-O National Liberation Organisation (PNLO)', 'National Socialist Council of Nagaland (NSCN)',
       'All Burma Students Democratic Front (ABSDF)', 'Kalo Htoo Baw faction of Democratic Karen Benevolent Army (DKBA-5)', 'Kachin Independence Organisation (KIO)',
       'Restoration Council of Shan State (RCSS)', 'Arakan National Council','Arakan Army', 'Lahu Democratic Union','New Mon State Party (NMSP)', 'Rakhine State Liberation Party (RSLP)','Ethnic Representative Ethnic Armed Organization Group',
       'Relevant Representative of Ethnic Armed Organization Group\xa0','Shan State Progress Party (SSPP)/Shan State Army (SSA)', 'Arakan Liberation Party', 'SSPP/SSA', 'Karenni National Progressive Party (KNPP)', ' NCA - S',' EAOs ',],
    'Civil Society':[ ' Chinland Guardian', ' Community Radio', ' Civilians', ' Civilian', ' Sky Net TV', ' Shan Herald', ' Tai Freedom',],
    'International NGOs and Intergovernmental Organizations':[' Non Governmental Organizations',
       ' International Non- Governmental Organizations',' NGOS','Food and Drug Administration (FDA)', ' Anti - Narcotics Development ',  ' CCDAC', ' CCDAC - UNODC', 
       ' Central Committee Drug Abuse Control',  'Non - Governmental Organizations', ],
    'United Nations':['United Nations', 'United Nations Office on Drugs and Crime (UNODC)'],
    'Political Party': [' Party', 'Political Parties Group',  'Arakan National Council', 
       ' Political Parties Group  Page',
       ' Political Parties Group  Union',
       ' Political Parties Registration Law Chapter II','Ethnic Representative Political Parties Group',
       'Relevant Representative of the Political Parties Group\xa0','National League Democracy (NLD)','Nationalities Brotherhood Federation (NBF)','United Nationalities Alliance (UNA)', 'United Nationalities Federal Council (UNFC)'],
    'Government Group': [' UWSA Government','Shan State Government','Karen State Government', 'Chin State Government',' Liaison Office Economic', ' Central Executive Committee ’s',' National Human Rights Commission', 'Union Election Commission (UEC)',' Leaders Executive', ' CBO',' Union Election Commission',       
       ' Electoral Management Body', ' Party Central Committee', ' Chin State Border Areas Security Affairs',], 
    'Military': ['Tatmadaw', ' Tamataw', ' Burma Army',' Army ) Office', ' Myanmar Army',' Formation Union Armed Forces',' Northwestern Commands', ' State Security', ' Ministry Security','Myanmar National Democratic Alliance Army',' State Army', ' Ministry Defense ( Army',],
    'Peace Committee': [   ' Peace Tranquillity Committee',' Kayin State Peace Making Group', ' Peace Team', ' Union Peacemaking Work',' Peacemaking Group', ' Internal Peace Committee',
       ' Ceasefire  Union',' Peace Dialogue Joint Committee', 'Wa Special Region (2) Peacemaking Group',' Implementing Union',' Peace Making Group',' MPC',' Work Committee', ' Peace Committee Burma',' Working Committees',' Peace Working Committee',' Ceasefire Monitoring Body ','Union Peace Dialogue Joint Committee (UPDJC)',' Peace Facilitating Committee', 'Joint Ceasefire Monitoring Committee (JCMC)','Joint Monitoring Committee (JMC)',
       'Union Level Joint Ceasefire Monitoring Committee (JMC-U)',' Union - Level', 'Union Peacemaking Working Committee (UPWC)', ' Working Committee', ' Monitoring Committees', 'State Level Joint Ceasefire Monitoring Committee (JMC-S)',
        ' Tranquility', ' Nationwide Ceasefire Agreement Joint Implementation Coordination Meetings', 'Joint Implementing Coordination Meeting (JICM)',
       'Local Level Joint Ceasefire Monitoring Committee (JMC-L)',' Chin Peace Monitoring Body', ' Chin Peace Tranquility Committee', ' Ceasefire Monitoring Body', ' Tranquility Committee',  ],
    'Government': ['Government of the Republic of the Union of Myanmar', ' Nationwide Ceasefire Coordination Team', ' NCCT',   ' Republic Union Myanmar Government', ' People ’s Parliament',
       ' National Parliament',  ' Union Ministries',   ' Union Ministers', ' National Liberation Organization Myanmar Government',  ' Pyithu Hluttaw Law', ' Taungtha Township Pyithu Hluttaw Representative', ' Amoytha Hluttaw Law', ' Immigration Department', ' Ministry Interior ',  ' Government Union Mynamar', 'Hluttaw', ' CoC Monitoring Committee', ' Parliament', ' Hluttaw Election',' Hluttaw Laws',  ], 
    'Country':['Myanmar (Burma)' ,' Thailand',],
    'Law':[' Unlawful Association Act', ' CoC',' Panglong Agreement',  ' Kachin Hill Tribes Regulation',
       ' Government Burma Act', ' Pyidaungsu Accord', ' Conduct', ' Establishment Union', 
       ' Constitutional Tribunal',
       ' Accordance', ' NDAA', ],
    'Location':[' Karen State', ' Eastern Shan',' Mon State', ' Panglong', ' Hakha', ' Chin State', ' Rangoon', ' Thantlang Township', ' Vanzang', ' Lungding', ' Belhar',
       ' Lulpilung Village Tracts', ' Tedim Township', ' Battalion', ' Naypyidaw',
       ' Yangon', ' Mandalay', ' Kalay', ' Sittwe', ' Seih',  ' Tonzang Township', ' Mela', ' Phayathonzu',
       ' Kawkareik', ' Mawtaung', ' Kawthoung',  ' Sittway', ' Kyauktaw ', ' Tachileik', ' Muse',
       ' Nanhkan',   
       ' Paletwa Township',' Zangtlang Village', ' Shan', ' Pansang',  ' Sukali',
       ' Zampi', ' Bukphir Village', ' Kung Pyin', ' Mauk Mae', ' Tha Htone', 
       ' Pathiantlang Village',  ' Thantlang Township ’s Belhar', ' Lungding Village', ' Lulpilung',
       ' Tedim Township ’s Dampi', ' Tui Dial Village',
       ' Shweletwa Village', ' Ra Kan Village', ' Myitkyina City',  ' Panlong',
       ' Kachin State', ' Kayan Township', ' Myitkyina Township', ' New Village Project',
        ' Tlangpi', ' Mongnai', ' Payathonsu',' Wanhai', ' Lashio',
       ' Kholan', ' Hsihseng', ' Maukmai', ' Myitkyina',' Hsee',' Khamti', ' Sagaing Region',
              ' Homain', ' Kyi Naing', 
         ' Mong Hta', ' Mong Hsat', ' Ka Du Gyi', ' Mae Aww',
        ' Kayah State', ' Naypyitaw', ' Naung', ' Century Panglong', ' Tedim',
       ' Mongpan', ' Thantlang', ' Matupi', ' Sonsee', ' Myawaddy',' Station Hall', 'Taunggyi', ' Kholam', ' Tachilek', ' Mongton',
       ' Thantlang Township :', ' Tlangpi Village Tract', ' Matupi Towns',' Kengtung', ' Strand Hotel Mawlamyine', 
       ' Dawn Village Tract', ' Zang Tlang Village Tract', ' Thanglang', ' South',
       ' Tedim Township ’s', ' Paletwa Township ’s Kung Pin',' Taungtha Township', ' Village', ], 
    'Peace Talks':[ ' Peace Talks ', ' Union Peace Conference', ' “ Common Agreement',' Panglong Conference',' National Reconciliation ',
       ' Deed Commitment Peace National Reconciliation', ' Nationwide Ceasefire Agreement Section 20',' Nationwide Ceasefire Agreement', ' NCA', ' CNF - Government Ceasefire Agreement', ' Ceasefire',],
    'Political Dialogue': [' “ Framework Political Dialogue', ' Political Dialogue',' Framework Political Dialogue',' Security  Sectors Reforms', ' Chapter 7 Organization Political Dialogue', ' National Political Dialogue'],
    'Police':[' Myanmar Police Force', ' Myanmar Army Police Force']   
}

In [31]:
def entity_type(e_type):
    for key, value in type_dict.items():
        if e_type in value:
            return key  

In [32]:
both['entity_overall_description']=both['entity_affiliated_org'].apply(entity_type)

In [33]:
type_dict['EAO']

['Ethnic Armed Organizations (EAO)',
 'Chin National Front (CNF)',
 'Karen National Union (KNU)/Karen National Liberation Army (KNLA)',
 'Democratic Karen Benevolent/Buddhist Army (DKBA)',
 'Pa-O National Liberation Organisation (PNLO)',
 'National Socialist Council of Nagaland (NSCN)',
 'All Burma Students Democratic Front (ABSDF)',
 'Kalo Htoo Baw faction of Democratic Karen Benevolent Army (DKBA-5)',
 'Kachin Independence Organisation (KIO)',
 'Restoration Council of Shan State (RCSS)',
 'Arakan National Council',
 'Arakan Army',
 'Lahu Democratic Union',
 'New Mon State Party (NMSP)',
 'Rakhine State Liberation Party (RSLP)',
 'Ethnic Representative Ethnic Armed Organization Group',
 'Relevant Representative of Ethnic Armed Organization Group\xa0',
 'Shan State Progress Party (SSPP)/Shan State Army (SSA)',
 'Arakan Liberation Party',
 'SSPP/SSA',
 'Karenni National Progressive Party (KNPP)',
 ' NCA - S',
 ' EAOs ']

In [34]:
nca_dict={
  'NCA-S':['All Burma Students Democratic Front (ABSDF)', 'Arakan National Council','Chin National Front (CNF)',
 'Karen National Union (KNU)/Karen National Liberation Army (KNLA)',
 'Democratic Karen Benevolent/Buddhist Army (DKBA)','Restoration Council of Shan State (RCSS)','Lahu Democratic Union',
 'New Mon State Party (NMSP)'],
    'Non-Signatory':['Arakan Army','Karenni National Progressive Party (KNPP)','Shan State Progress Party (SSPP)/Shan State Army (SSA)','SSPP/SSA']
}

In [35]:
def nca_status(status):
    for key, value in nca_dict.items():
        if status in value:
            return key  

In [36]:
both['NCA_status']=both['entity_affiliated_org'].apply(nca_status)

In [37]:
both.to_csv('Myanmar_entities_labelled_types.csv')

## Nodes and relationships 
Possibilities for defining nodes and relationships between them:
1. Each entity (row) is a node, and each peace agreement is a node => they are related if the entity is mentioned in the agreement. This can be weighted by how many mentions of them in that agreement.
2. Each unique entity is a node. They are related to other nodes which are the same affiliated organisation. 
3. Each unique entity is a node. They are related to other nodes that are the same type of entity (description of entity). 

In [38]:
agreements=both.groupby('agreement_id')

In [39]:
agreement_counts=agreements['entity_named'].value_counts()
df1=pd.DataFrame(agreement_counts)

In [40]:
df1=df1.rename(columns={'entity_named':'mentions_per_agreement'}).reset_index()

In [41]:
df1.to_csv('entities/entity_mentions_per_agreement.csv')

In [78]:
df1['agreement_id']=df1['agreement_id'].astype(str)

In [73]:
G = networkx.from_pandas_edgelist(df1, 'agreement_id','entity_named', 'mentions_per_agreement')

In [42]:
orgs=both.groupby('entity_affiliated_org')

In [43]:
unique_entity=both['entity_named'].unique()

In [44]:
list(both)

['agreement_id',
 'entity_type',
 'entity_named',
 'entity_affiliated_org',
 'entity_affiliated_org_abr',
 'NER',
 'entity_overall_description',
 'NCA_status']

In [45]:
df_e=both[['entity_named',
 'entity_affiliated_org',
 'entity_affiliated_org_abr','entity_overall_description',
 'NCA_status']]

In [46]:
unique_ents=df_e.drop_duplicates()

In [47]:
unique_ents.to_csv('entities/unique_named_entities_desc.csv')

In [48]:
unique_ents

,entity_named,entity_affiliated_org,entity_affiliated_org_abr,entity_overall_description,NCA_status
0,Chin National Front (CNF),Chin National Front (CNF),CNF,EAO,NCA-S
1,Chinland Guardian,Chinland Guardian,None,Civil Society,None
3,Peace Committee Burma,Peace Committee Burma,None,Peace Committee,None
4,Rangoon,Rangoon,None,Location,None
6,Peace Working Committee,Peace Working Committee,None,Peace Committee,None
...,...,...,...,...,...
13,U Htone Phoo Dagon,Kachin National Association of Tradition and C...,Gov. of Myanmar,None,None
14,Kya Sha Mwe,Ethnic Representative Ethnic Armed Organizatio...,EAO,EAO,None
15,Daw Khin Ma Ma Myo,Relevant Representative of Ethnic Armed Organi...,EAO,EAO,None
16,U Zein Khan Ram,Ethnic Representative Political Parties Group,Political Parties Group,Political Party,None


In [49]:
df2=unique_ents[['entity_named','entity_affiliated_org']]

In [50]:
df2['relation_type']='affiliated with'

C:\Users\nhenry2\AppData\Local\Temp/ipykernel_13308/1447830439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['relation_type']='affiliated with'


In [51]:
list(df2)

['entity_named', 'entity_affiliated_org', 'relation_type']

In [52]:
df2=df2[['entity_named','relation_type', 'entity_affiliated_org']]

In [53]:
df2.to_csv('entity_linked_affiliated_org.csv')

In [56]:
def clean(s):
    if (s['entity_named'] == s['entity_affiliated_org']):
        return 'same'

In [59]:
df2['org_afil']=df2.apply(clean, axis=1)

In [64]:
df2_cleaned=pd.DataFrame(df2.loc[df2['org_afil']!='same'])

In [66]:
df2_cleaned=df2_cleaned.drop(columns=['org_afil'])

In [68]:
df2_cleaned.to_csv('entities/known_org_affiliations.csv')

In [69]:
df2_cleaned

,entity_named,relation_type,entity_affiliated_org
35,General Aung San,affiliated with,Government of the Republic of the Union of Mya...
418,Lt. Gen. Ya Pyit,affiliated with,Government of the Republic of the Union of Mya...
419,U Khin Maung Soe,affiliated with,Government of the Republic of the Union of Mya...
420,Lt. Gen. Ye Aung,affiliated with,Government of the Republic of the Union of Mya...
421,Maj. Gen. Tun Tun Naung,affiliated with,Government of the Republic of the Union of Mya...
...,...,...,...
13,U Htone Phoo Dagon,affiliated with,Kachin National Association of Tradition and C...
14,Kya Sha Mwe,affiliated with,Ethnic Representative Ethnic Armed Organizatio...
15,Daw Khin Ma Ma Myo,affiliated with,Relevant Representative of Ethnic Armed Organi...
16,U Zein Khan Ram,affiliated with,Ethnic Representative Political Parties Group


In [99]:
unique_ents

,entity_named,entity_affiliated_org,entity_affiliated_org_abr,entity_overall_description,NCA_status
0,Chin National Front (CNF),Chin National Front (CNF),CNF,EAO,NCA-S
1,Chinland Guardian,Chinland Guardian,None,Civil Society,None
3,Peace Committee Burma,Peace Committee Burma,None,Peace Committee,None
4,Rangoon,Rangoon,None,Location,None
6,Peace Working Committee,Peace Working Committee,None,Peace Committee,None
...,...,...,...,...,...
13,U Htone Phoo Dagon,Kachin National Association of Tradition and C...,Gov. of Myanmar,None,None
14,Kya Sha Mwe,Ethnic Representative Ethnic Armed Organizatio...,EAO,EAO,None
15,Daw Khin Ma Ma Myo,Relevant Representative of Ethnic Armed Organi...,EAO,EAO,None
16,U Zein Khan Ram,Ethnic Representative Political Parties Group,Political Parties Group,Political Party,None


In [85]:
prop=unique_ents.groupby('entity_overall_description')

In [94]:
p=prop['entity_named'].unique()

In [95]:
p1=pd.DataFrame(p)

In [97]:
p1=p1.reset_index()

In [98]:
p1

,entity_overall_description,entity_named
0,Civil Society,"[ Chinland Guardian, Community Radio, Civili..."
1,Country,"[Myanmar (Burma), Thailand]"
2,EAO,"[Chin National Front (CNF), Ethnic Armed Organ..."
3,Government,"[General Aung San, Government of the Republic ..."
4,Government Group,"[ National Human Rights Commission, Leaders E..."
5,International NGOs and Intergovernmental Organ...,"[ Non Governmental Organizations, Internation..."
6,Law,"[ Panglong Agreement, Kachin Hill Tribes Regu..."
7,Location,"[ Rangoon, Panglong, Hakha, Chin State, Th..."
8,Military,"[ Burma Army, Tatmadaw, Tamataw, Ministry De..."
9,Peace Committee,"[ Peace Committee Burma, Peace Working Commit..."
